# Project Name

## Visual Data Science

Clara Pichler, 11917694

### Overview

1. Introduction
- Original Research Questions
- Used Datasets
- Requirements & Dependencies

2. Data Preprocessing
- Loading in the Data Sets
- First Steps
- Missing Values
- Merging Data Sets

3. Data Exploration
- 

4. Conclusion

### Notes

Bis 16.12

__Wrangle:__ 
- Join the two or more datasets you selected into one big data table, and
- Solve issues like formatting issues, missing data, faulty values, and non-matching keys
- _10 points_ extra, if you can (visually!) show and explain the data quality of your dataset (for example, before and after cleaning steps). There are no instructions how such a visualization should look like, since this also highly depends on the chosen domain and data. You will have to come up with your own, creative, solution here.

__Profile:__
- Find at least 3 valuable insights.
- Show the insights by using visualization.

In the Profile part, you will have to explore the data in detail, to completely understand its structure, and to discover any interesting patterns that can be found in there. The following steps are required:

- Find at least 3 informative insights in your dataset, and
- Show and explain the insights using visualizations (one visualization per insight).



## Introduction

### Original Research Questions

- Are there any similarities between countries which win medals often in specific kinds of sports?
- Which factors influence the amount of medals a country wins? 
- Is it possible to predict the number of won medals of a country?
- Are there any trends in the medal distribution over time?
- Which factors play a part in the "under-dog" countries winning suddenly some medals?


### Used Data Sets

__Olympics Medals Data Set__
- _description:_ General information on medalists (Athlete or Team)
- _source:_ https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018
- _note:_ This data set comes with three other data sets that include information on the athletes, hosts and all of the results. I will only focus on the medals so first, second and third place, therefore only using the `olympic_medals.csv` data set. 
- _disadvantages:_ Quote from the description
    - "There are no results for qualification rounds. For instance, event 100-m men contains only final results without semi-finals and other hits."
    - "There is no information about athletes for team competitions that consist of more than 2 participants. Only team records."

__Rugged Data Set__
- _description:_
- _source:_ https://diegopuga.org/data/rugged/ 
- 

__Elevation Extremes Data Set__
- _description:_ A sortable table which lists land surface elevation extremes by country or dependent territory
- _source:_ https://en.wikipedia.org/wiki/List_of_elevation_extremes_by_country 
- 

### Requirements & Dependencies

This project was created using Python 3.12.5. The exact versions of the dependencies can be installed with the following command.

In [ ]:
%pip install -r requirements.txt -q

In [70]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as r
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

## Data Pre-Processing

### Loading in the Data Sets

First and foremost we will load in our data sets. For the rugged data I had some problems with encoding since two countries have special characters in their name, as one can see below. By using `encoding='latin1'` this issue was able to be resolved.

For the elevation data set which I got from Wikipedia, I used `BeautifulSoup` to scrape the table from their website. I treid firts `pd.read_html` however this simple approach did not work for me like it did for others.

In [ ]:
with open('data/rugged_data/rugged_data.csv', 'r', encoding='ascii', errors='replace') as f:
    for i, line in enumerate(f):
        if '�' in line:  
            print(f"Problem in row {i + 1}: {line}")

In [151]:
df_olympics = pd.read_csv('data/olympics/olympic_medals.csv')
df_rugged = pd.read_csv('data/rugged_data/rugged_data.csv', encoding='latin1')

In [71]:
url = "https://en.wikipedia.org/wiki/List_of_elevation_extremes_by_country"
response = r.get(url)
wiki_page_text = response.text

In [72]:
response.status_code # 200 means everything is A-OK

200

In [144]:
soup = BeautifulSoup(wiki_page_text, 'html.parser')
elevation_table = soup.find('table',{'class':'wikitable sortable'})

headers = [header.text.strip() for header in elevation_table.find_all('th')]

rows = []
for row in elevation_table.find_all('tr')[1:]:  
    cells = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
    rows.append(cells)

df_elevation = pd.DataFrame(rows, columns=headers)

### First Steps

First we will take a first look at the `df_olympics` dataframe. We will drop unnecessary columns like `athlete_url` and `country_code` since we only need the ISO code which is the `country_3_letter_code` attribute. I do not care about the host country so I changed the `slug_game` attribute to `year` and left out the host name. The type was changed to `int`, all other types are fine. Maybe we will change `medal_type` gold, silver and bronze to 1, 2, and 3 in the future for making it easier to work with. 

I also drop the column `athlete_full_name` which will result in rows that are not unique since for events with the `participant_type` `GameTeam` we have a medal entry for each athlete in the team. Thats why I use `drop_duplicates()`.


In [152]:
df_olympics = df_olympics.drop(['athlete_url', 'country_code', 'athlete_full_name'], axis=1).drop_duplicates()

df_olympics = df_olympics.rename(columns={
    'slug_game': 'year',
    'country_3_letter_code': 'isocode'
})

df_olympics['year'] = df_olympics['year'].str[-4:].astype(int)

df_olympics = df_olympics.reset_index(drop=True)


In [153]:
display(df_olympics.head(5))
display(df_olympics.info())
print(df_olympics['year'].unique())

,discipline_title,year,event_title,event_gender,medal_type,participant_type,participant_title,country_name,isocode
0,Curling,2022,Mixed Doubles,Mixed,GOLD,GameTeam,Italy,Italy,ITA
1,Curling,2022,Mixed Doubles,Mixed,SILVER,GameTeam,Norway,Norway,NOR
2,Curling,2022,Mixed Doubles,Mixed,BRONZE,GameTeam,Sweden,Sweden,SWE
3,Curling,2022,Women,Women,GOLD,GameTeam,Great Britain,Great Britain,GBR
4,Curling,2022,Women,Women,SILVER,GameTeam,Japan,Japan,JPN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20178 entries, 0 to 20177
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   discipline_title   20178 non-null  object
 1   year               20178 non-null  int64 
 2   event_title        20178 non-null  object
 3   event_gender       20178 non-null  object
 4   medal_type         20178 non-null  object
 5   participant_type   20178 non-null  object
 6   participant_title  5104 non-null   object
 7   country_name       20178 non-null  object
 8   isocode            20178 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.4+ MB


None

[2022 2020 2018 2016 2014 2012 2010 2008 2006 2004 2002 2000 1998 1996
 1994 1992 1988 1984 1980 1976 1972 1968 1964 1960 1956 1952 1948 1936
 1932 1928 1924 1920 1912 1908 1904 1900 1896]


Moving on with `df_elevation`. The attributes `Continent`, `Highest point` and `Lowest point` ist non of my interest from this data set

In [145]:
df_elevation = df_elevation.drop(['Continent', 'Highest point', 'Lowest point'], axis=1)

df_elevation = df_elevation.rename(columns={
    'Country or region': 'country_name',
    'Maximum elevation': 'max_elevation',
    'Minimum elevation': 'min_elevation',
    'Elevation span' : 'elevation_span'
})

In [146]:
df_elevation = df_elevation.replace("sea level", "0")

for col in ['max_elevation', 'min_elevation', 'elevation_span']: 
    df_elevation[col] = (df_elevation[col].str.split('m')
                         .str[0].str.strip()
                         .str.replace('−', '-', regex=False)
                         .str.replace(',', '.', regex=False))
    df_elevation.replace({col : ''}, np.nan, inplace=True)
    df_elevation[col] = df_elevation[col].astype(float)

In [147]:
display(df_elevation.head(5))


,country_name,max_elevation,min_elevation,elevation_span
0,Afghanistan,7492.0,258.0,7234.0
1,Albania,2764.0,0.0,2764.0
2,Algeria,3003.0,-40.0,3043.0
3,American Samoa,966.0,0.0,966.0
4,Andorra,2942.0,840.0,2102.0


Moving forward to `df_rugged`. It has so much data and so much potential. I will only use some of the many attributes it provides so that this project won't take forever. 

In [ ]:
df_rugged = df_rugged[['isocode', 'rugged', 'rugged_popw', 'rugged_slope',
       'rugged_lsd', 'rugged_pc', 
       'land_area', # Units are thousands of hectares
       'lat', 'lon', 'soil', 
       'desert', # % Desert: sandy desert, dunes, rocky or lava flows
       'tropical', # % Tropical climate
       'dist_coast', 'near_coast', # % Within 100 km of ice-free coast
       'gemstones', 'rgdppc_2000',
       'rgdppc_1950_m', 'rgdppc_1975_m', 'rgdppc_2000_m', 'rgdppc_1950_2000_m',
       'q_rule_law', 'cont_africa', 'cont_asia', 'cont_europe', 'cont_oceania',
       'cont_north_america', 'cont_south_america', 'legor_gbr', 'legor_fra',
       'legor_soc', 'legor_deu', 'legor_sca', 'colony_esp', 'colony_gbr',
       'colony_fra', 'colony_prt', 'colony_oeu', 'africa_region_n',
       'africa_region_s', 'africa_region_w', 'africa_region_e',
       'africa_region_c', 'slave_exports', 'dist_slavemkt_atlantic',
       'dist_slavemkt_indian', 'dist_slavemkt_saharan', 'dist_slavemkt_redsea',
       'pop_1400', 'european_descent']]

In [54]:
display(df_rugged.head(5))
display(df_rugged.info())
display(df_rugged.columns)

,isocode,isonum,country,rugged,rugged_popw,rugged_slope,rugged_lsd,rugged_pc,land_area,lat,...,africa_region_w,africa_region_e,africa_region_c,slave_exports,dist_slavemkt_atlantic,dist_slavemkt_indian,dist_slavemkt_saharan,dist_slavemkt_redsea,pop_1400,european_descent
0,ABW,533,Aruba,0.462,0.380,1.226,0.144,0.000,18.0,12.508,...,0,0,0,0.0,NaN,NaN,NaN,NaN,614.0,NaN
1,AFG,4,Afghanistan,2.518,1.469,7.414,0.720,39.004,65209.0,33.833,...,0,0,0,0.0,NaN,NaN,NaN,NaN,1870829.0,0.0
2,AGO,24,Angola,0.858,0.714,2.274,0.228,4.906,124670.0,-12.299,...,0,0,1,3610000.0,5.669,6.981,4.926,3.872,1223208.0,2.0
3,AIA,660,Anguilla,0.013,0.010,0.026,0.006,0.000,9.0,18.231,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ALB,8,Albania,3.427,1.597,10.451,1.006,62.133,2740.0,41.143,...,0,0,0,0.0,NaN,NaN,NaN,NaN,200000.0,100.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   isocode                 234 non-null    object 
 1   isonum                  234 non-null    int64  
 2   country                 234 non-null    object 
 3   rugged                  234 non-null    float64
 4   rugged_popw             234 non-null    float64
 5   rugged_slope            234 non-null    float64
 6   rugged_lsd              234 non-null    float64
 7   rugged_pc               234 non-null    float64
 8   land_area               230 non-null    float64
 9   lat                     234 non-null    float64
 10  lon                     234 non-null    float64
 11  soil                    225 non-null    float64
 12  desert                  234 non-null    float64
 13  tropical                234 non-null    float64
 14  dist_coast              234 non-null    fl

None

Index(['isocode', 'isonum', 'country', 'rugged', 'rugged_popw', 'rugged_slope',
       'rugged_lsd', 'rugged_pc', 'land_area', 'lat', 'lon', 'soil', 'desert',
       'tropical', 'dist_coast', 'near_coast', 'gemstones', 'rgdppc_2000',
       'rgdppc_1950_m', 'rgdppc_1975_m', 'rgdppc_2000_m', 'rgdppc_1950_2000_m',
       'q_rule_law', 'cont_africa', 'cont_asia', 'cont_europe', 'cont_oceania',
       'cont_north_america', 'cont_south_america', 'legor_gbr', 'legor_fra',
       'legor_soc', 'legor_deu', 'legor_sca', 'colony_esp', 'colony_gbr',
       'colony_fra', 'colony_prt', 'colony_oeu', 'africa_region_n',
       'africa_region_s', 'africa_region_w', 'africa_region_e',
       'africa_region_c', 'slave_exports', 'dist_slavemkt_atlantic',
       'dist_slavemkt_indian', 'dist_slavemkt_saharan', 'dist_slavemkt_redsea',
       'pop_1400', 'european_descent'],
      dtype='object')

### Missing Values

### Merging Data Sets

## Data Exploration

